In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from pathlib import Path

# --- 0. Configuration et Définition des Chemins ---
print("🚀 Configuration de l'application RAG...")

# Définir le chemin racine du projet (ajuster si nécessaire)
current_path = Path.cwd()
if current_path.name == "notebooks":
    PROJECT_ROOT = current_path.parent
else:
    PROJECT_ROOT = current_path
    
PROCESSED_DATA_PATH = PROJECT_ROOT / "data" / "processed"
FAISS_INDEX_PATH = PROCESSED_DATA_PATH / "my_documents.index"
CHUNKS_PATH = PROCESSED_DATA_PATH / "my_documents_chunks.pkl"

# --- 1. Chargement des Modèles ---
print("🧠 Chargement des modèles (LLM et Embedding)...")

# Charger le LLM (Phi-3)
llm_model_id = "microsoft/Phi-3-mini-4k-instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id, trust_remote_code=True)

# Charger le modèle d'Embedding
embedding_model = SentenceTransformer('BAAI/bge-small-en-v1.5', device='cpu')
print("✅ Modèles chargés.")


# --- 2. Chargement des Artefacts RAG (le "cerveau") ---
print("📚 Chargement de l'index FAISS et des chunks de texte...")
try:
    index = faiss.read_index(str(FAISS_INDEX_PATH))
    with open(CHUNKS_PATH, "rb") as f:
        chunks = pickle.load(f)
    print("✅ Index et chunks chargés avec succès.")
except Exception as e:
    print(f"❌ Erreur lors du chargement des fichiers RAG. Avez-vous exécuté le script de création d'index d'abord ?")
    print(f"Erreur : {e}")
    # Arrêter le script si les fichiers ne peuvent pas être chargés
    exit()

# --- 3. Définition de la fonction RAG principale ---

def answer_question_with_rag(question: str, k: int = 5) -> str:
    """
    Prend une question, trouve les chunks pertinents, construit un prompt et génère une réponse.
    """
    
    # Étape 3.1 : Vectoriser la question de l'utilisateur
    question_embedding = embedding_model.encode([question])
    
    # Étape 3.2 : Chercher dans l'index FAISS
    distances, indices = index.search(question_embedding.astype('float32'), k)
    
    # Étape 3.3 : Récupérer les chunks de texte pertinents
    retrieved_chunks = [chunks[i] for i in indices[0]]
    context_text = "\n\n---\n\n".join(retrieved_chunks)
    
    # Étape 3.4 : Construire le prompt pour le LLM
    # C'est ici que le "prompt engineering" est crucial.
    prompt_template = f"""
<|system|>
Vous êtes un assistant expert qui répond aux questions de manière précise en vous basant sur le contexte fourni ci-dessous.
.
Ne mentionnez pas l'existence du "contexte" dans votre réponse. Répondez directement à la question.

Contexte fourni :
{context_text}<|end|>
<|user|>
{question}<|end|>
<|assistant|>
"""
    
    # Étape 3.5 : Générer la réponse avec le LLM
    input_ids = llm_tokenizer(prompt_template, return_tensors="pt").to(llm_model.device)
    
    outputs = llm_model.generate(
        **input_ids,
        max_new_tokens=512,  # On peut augmenter un peu pour des réponses plus longues
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        eos_token_id=llm_tokenizer.eos_token_id
    )
    
    # Étape 3.6 : Décoder la réponse
    full_response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraire uniquement la partie générée par l'assistant
    # Cette méthode est plus simple et robuste
    assistant_part = full_response.split("<|assistant|>")
    if len(assistant_part) > 1:
        return assistant_part[1].strip()
    else:
        # Fallback si le formatage est inattendu
        return "Désolé, une erreur est survenue lors de la génération de la réponse."


# --- 4. Boucle de Chat Interactive ---
if __name__ == "__main__":
    print("\n" + "="*50)
    print("🤖 Assistant RAG prêt. Posez vos questions sur les documents.")
    print("Tapez '/exit' pour quitter.")
    print("="*50 + "\n")

    while True:
        user_question = input("Vous: ")
        if user_question.lower() == '/exit':
            break
        
        # Obtenir la réponse via la pipeline RAG
        answer = answer_question_with_rag(user_question)
        
        print(f"Assistant: {answer}\n")

    print("👋 Au revoir !")

🚀 Configuration de l'application RAG...
🧠 Chargement des modèles (LLM et Embedding)...


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/elyes/miniconda3/envs/llm-dev/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Modèles chargés.
📚 Chargement de l'index FAISS et des chunks de texte...
✅ Index et chunks chargés avec succès.

🤖 Assistant RAG prêt. Posez vos questions sur les documents.
Tapez '/exit' pour quitter.



Vous:  donne moi l'importance ou l'objectif de la Gestion de version de code


You are not running the flash-attention implementation, expect numerical differences.


Assistant: Désolé, une erreur est survenue lors de la génération de la réponse.



In [4]:
import torch
import gc

# Delete large objects

# del model
# del tokenizer
# del embeddings
del input_ids
del outputs
del question_embedding

# Force Python's garbage collection
gc.collect()

# Empty PyTorch's CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("VRAM cache cleared.")

NameError: name 'input_ids' is not defined